<a href="https://colab.research.google.com/github/sharmapratik88/LearningAI/blob/main/NLP_HuggingFace_LLMs/Text_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
# Setting the current working directory
import os; os.chdir('/content/drive/MyDrive/Learnings/Misc/')

In [3]:
# !nvidia-smi

In [4]:
!pip install -q transformers[sentencepiece]
!pip install -q datasets
!pip install -q sacrebleu rouge_score
!pip install -q py7zr
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install -q transformers accelerate

Found existing installation: transformers 4.38.1
Uninstalling transformers-4.38.1:
  Successfully uninstalled transformers-4.38.1
Found existing installation: accelerate 0.27.2
Uninstalling accelerate-0.27.2:
  Successfully uninstalled accelerate-0.27.2


In [5]:
# Importing
import pandas as pd
import matplotlib.pyplot as plt
import torch
import nltk

from nltk.tokenize import sent_tokenize
from tqdm import tqdm
nltk.download("punkt")

# model download
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset, load_from_disk, load_metric

# finetuning
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
device="gpu" if torch.cuda.is_available() else "cpu"
print(device)

gpu


In [7]:
model="google/pegasus-cnn_dailymail"
tokenizer=AutoTokenizer.from_pretrained(model)
model_pegasus=AutoModelForSeq2SeqLM.from_pretrained(model).to('cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [8]:
dataset=load_dataset("samsum")
dataset

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [9]:
# train dataset
train = dataset['train']

# dialogues in train dataset
dialogue = train['dialogue']

# summary in train dataset
summary = train['summary']

print('Fetch first dialogue and summary')
print(dialogue[0], summary[0])

Fetch first dialogue and summary
Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-) Amanda baked cookies and will bring Jerry some tomorrow.


In [10]:
def convert_examples_to_features(data_in_batch):
    # Tokenize the dialogues in the batch
    input_encoding = tokenizer(data_in_batch['dialogue'],
                               max_length=1024,
                               truncation=True)

    # Tokenize the summaries in the batch
    target_encoding = tokenizer(data_in_batch['summary'],
                                max_length=128,
                                truncation=True)

    # Return a dictionary containing input and target tokenized sequences
    return {
        'input_ids': input_encoding['input_ids'],  # Input token IDs for dialogues
        'attention_mask': input_encoding['attention_mask'],  # Attention mask for dialogue inputs
        'labels': target_encoding['input_ids']  # Target token IDs for summaries (used as labels)
    }

In [11]:
dataset_en = dataset.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [12]:
# Display the first example from the training dataset
display(
    dataset_en['train'],  # Display the training dataset
    dataset_en["train"]['input_ids'][0],  # Display the input token IDs of the first example
    dataset_en['train']['attention_mask'][0],  # Display the attention mask of the first example
    dataset_en['train']['labels'][0]  # Display the target token IDs (labels) of the first example
)

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

[12195,
 151,
 125,
 7091,
 3659,
 107,
 842,
 119,
 245,
 181,
 152,
 10508,
 151,
 7435,
 147,
 12195,
 151,
 125,
 131,
 267,
 650,
 119,
 3469,
 29344,
 1]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

[12195, 7091, 3659, 111, 138, 650, 10508, 181, 3469, 107, 1]

In [ ]:
%%timeit
# reference: https://huggingface.co/docs/transformers/v4.38.1/en/main_classes/trainer#transformers.Seq2SeqTrainingArguments
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args=TrainingArguments(
    output_dir='pegasus-samsum',
    num_train_epochs=1,
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=500,
    save_steps=1e6,
    gradient_accumulation_steps=16
)
trainer=Trainer(model=model_pegasus,
                args=training_args,
                tokenizer=tokenizer,
                data_collator=seq2seq_data_collator,
                train_dataset=dataset_en['train'],
                eval_dataset=dataset_en['validation'])

trainer.train()

Step,Training Loss,Validation Loss
500,1.659900,1.483296


In [ ]:
# Save the Pegasus model to the specified directory with the name 'pegasus-samsum-model'
model_pegasus.save_pretrained('pegasus-samsum-model')

# Save the tokenizer used with the Pegasus model to the specified directory with the name 'samsum-tokenizer'
tokenizer.save_pretrained('samsum-tokenizer')

In [ ]:
# Load tokenizer and model from the pretrained directories
tokenizer = AutoTokenizer.from_pretrained('samsum-tokenizer')
model = AutoModelForSeq2SeqLM.from_pretrained('pegasus-samsum-model')

# Create a pipeline object for summarization using the loaded model and tokenizer
pipeline_obj = pipeline('summarization', model=model, tokenizer=tokenizer)

# Define a sample text for summarization
sample_text_for_text = dataset['train']['dialogue'][2]
actual_summary = dataset['train']['summary'][2]
print(f'Sample text: {sample_text_for_text}')

In [ ]:
# Generate a summary using the pipeline object
gen_kwargs = {'length_penalty': 0.8, 'num_beams':8, 'max_length': 128}
display(pipeline_obj(sample_text_for_text, **gen_kwargs))
prediction = pipeline_obj(sample_text_for_text, **gen_kwargs)[0]['summary_text']

In [ ]:
# Evaluation
# evalation of model
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace('', ' ') for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [ ]:
rouge_names=['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
rouge_metric = load_metric('rouge')

In [ ]:
score = calculate_metric_on_test_ds(dataset['test'][0:10],
                                    rouge_metric,
                                    trainer.model,
                                    tokenizer,
                                    batch_size = 2,
                                    column_text = 'dialogue',
                                    column_summary= 'summary'
                                    )

rouge_dict = dict( (rn, score[rn].mid.fmeasure) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )